In [7]:
from sklearn.ensemble import VotingRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import roc_auc_score,accuracy_score,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression,ElasticNet

import warnings
warnings.filterwarnings("ignore")

In [8]:
med = pd.read_csv('insurance.csv')
X = med.drop('charges',axis=1)
y = med['charges']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=24, test_size=0.3)

In [10]:

ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe,make_column_selector(dtype_include=object)  ),
                             verbose_feature_names_out=False).set_output(transform='pandas')


In [13]:
dtr = DecisionTreeRegressor(random_state=24, max_depth=3)
pipe_dtr  = Pipeline([('CT',ct),('DT',dtr)])
lr= LinearRegression()
pipe_lr = Pipeline([('CT',ct),('LR',lr)])
el= ElasticNet()
pipe_el  = Pipeline([('CT',ct),('EL',el)])


In [15]:
pipe_dtr.fit(X_train,y_train)
y_pred = pipe_dtr.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)


0.8684500380700454


In [17]:
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)


0.7665391799816873


In [18]:
pipe_el.fit(X_train,y_train)
y_pred = pipe_el.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)




0.49673651151300835


Voting

In [19]:
voting = VotingRegressor([('LR',pipe_lr), ('EL',pipe_el),('DT', pipe_dtr)])
voting.fit(X_train,y_train)
y_pred= voting.predict(X_test)
r2_score(y_test,y_pred)

0.7931324996351616

Weighted Average

In [20]:
voting = VotingRegressor([('LR',pipe_lr), ('EL',pipe_el),('DT', pipe_dtr)],
                        weights= [0.4,0.2,0.4])
voting.fit(X_train,y_train)
y_pred= voting.predict(X_test)
r2_score(y_test,y_pred)

0.8239580766157142